In [ ]:
!pip install transformers[torch]

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import re

In [ ]:
data = pd.read_csv("nasias_effect_classification_file.csv")
data

,sentence,predicted_relation,source_name,target_name,eval_x,eval_y,eval_z,suggested_relation,suggested_source,suggested_target,...,eval_y.1,sent_id,pmid,source_uid,target_uid,source_tuis,target_tuis,source_span,target_span,db
0,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,1,1,NaN,NaN,NaN,NaN,...,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
1,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,1,1,NaN,NaN,NaN,NaN,...,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,GENE01,T046,[384 395],[453 465],oie_gene_dis
2,". Compared to NEC group, lactoferrin-treated N...",treats,lactoferrin,inflammation,1,1,NaN,NaN,NaN,NaN,...,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
3,[Absorption of fat from three dietary sources ...,causes,lactose,diarrhea,1,1,NaN,NaN,NaN,NaN,...,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
4,[Absorption of fat from three dietary sources ...,augments,lactose,diarrhea,1,1,NaN,NaN,NaN,NaN,...,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,Oral administration of lactic acid bacteria (L...,prevents,lactic acid bacteria,respiratory diseases,1,1,NaN,NaN,NaN,NaN,...,Iwannis Charitos,2.0,20163592,C1210581,C0035242,ORG02,T047,[23 43],[119 139],NaN
2267,We previously demonstrated that administration...,prevents,Akkermansia muciniphila,obesity,1,1,NaN,NaN,NaN,NaN,...,Iwannis Charitos,2.0,27892954,C1490590,C0028754,ORG02,T047,[50 73],[110 117],NaN
2268,The phage can also prevent abscess formation a...,prevents,phage,abscess,1,1,NaN,NaN,NaN,NaN,...,Iwannis Charitos,4.0,17517843,C0004651,C0000833,ORG03,T047,[4 9],[27 34],NaN
2269,"Thus, this study addressed whether GMI could b...",prevents,GMI,TBI,0,1,NaN,associated_with,GMI,TBI,...,Iwannis Charitos,6.0,35562037,C1005127,C0876926,ORG05,T037,[35 38],[73 76],NaN


In [ ]:
columns_to_drop = ['eval_x', 'eval_y', 'eval_z', 'suggested_relation', 'Biomarker', 'Endogenous']
new_data = data.drop(columns = columns_to_drop)
new_data

,sentence,predicted_relation,source_name,target_name,suggested_source,suggested_target,Effect Classification,Notes,dataset,eval_x.1,eval_y.1,sent_id,pmid,source_uid,target_uid,source_tuis,target_tuis,source_span,target_span,db
0,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
1,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,GENE01,T046,[384 395],[453 465],oie_gene_dis
2,". Compared to NEC group, lactoferrin-treated N...",treats,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
3,[Absorption of fat from three dietary sources ...,causes,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
4,[Absorption of fat from three dietary sources ...,augments,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,Oral administration of lactic acid bacteria (L...,prevents,lactic acid bacteria,respiratory diseases,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,20163592,C1210581,C0035242,ORG02,T047,[23 43],[119 139],NaN
2267,We previously demonstrated that administration...,prevents,Akkermansia muciniphila,obesity,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,27892954,C1490590,C0028754,ORG02,T047,[50 73],[110 117],NaN
2268,The phage can also prevent abscess formation a...,prevents,phage,abscess,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,4.0,17517843,C0004651,C0000833,ORG03,T047,[4 9],[27 34],NaN
2269,"Thus, this study addressed whether GMI could b...",prevents,GMI,TBI,GMI,TBI,Beneficial,No results are given,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,6.0,35562037,C1005127,C0876926,ORG05,T037,[35 38],[73 76],NaN


In [ ]:
# drop NaN rows from EC column
def drop_na(data):
  return data.dropna(subset = ['Effect Classification'])

new_data = drop_na(new_data)

In [ ]:
def process_data(data):
  data = drop_na(data)
  # set to empty string
  data.loc[(data['suggested_source'].notna()) & (data['suggested_source'] != data['source_name']), 'source_span'] = " "
  data.loc[(data['suggested_target'].notna()) & (data['suggested_target'] != data['target_name']), 'target_span'] = " "

# replace source_name and target_name if it's not NaN
  data.loc[data['suggested_source'].notna(), 'source_name'] = data['suggested_source']
  data.loc[data['suggested_target'].notna(), 'target_name'] = data['suggested_target']

  return data

new_data = process_data(new_data)
new_data


,sentence,predicted_relation,source_name,target_name,suggested_source,suggested_target,Effect Classification,Notes,dataset,eval_x.1,eval_y.1,sent_id,pmid,source_uid,target_uid,source_tuis,target_tuis,source_span,target_span,db
0,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
1,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,GENE01,T046,[384 395],[453 465],oie_gene_dis
2,". Compared to NEC group, lactoferrin-treated N...",treats,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,[384 395],[453 465],oie_chem_dis
3,[Absorption of fat from three dietary sources ...,causes,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
4,[Absorption of fat from three dietary sources ...,augments,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,[59 66],[75 83],oie_chem_dis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,Oral administration of lactic acid bacteria (L...,prevents,lactic acid bacteria,respiratory diseases,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,20163592,C1210581,C0035242,ORG02,T047,[23 43],[119 139],NaN
2267,We previously demonstrated that administration...,prevents,Akkermansia muciniphila,obesity,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,27892954,C1490590,C0028754,ORG02,T047,[50 73],[110 117],NaN
2268,The phage can also prevent abscess formation a...,prevents,phage,abscess,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,4.0,17517843,C0004651,C0000833,ORG03,T047,[4 9],[27 34],NaN
2269,"Thus, this study addressed whether GMI could b...",prevents,GMI,TBI,GMI,TBI,Beneficial,No results are given,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,6.0,35562037,C1005127,C0876926,ORG05,T037,[35 38],[73 76],NaN


In [ ]:
def parse_span(span):
  if isinstance(span, str) and span: #non-empty string
    if "|" in span:
      return list(map(int, span.split('|'))) # map substring to int in a list
    elif '[' in span and ']' in span:
      clean_span = span.replace('[', '').replace(']', '').replace(' ', ',').strip(',')
      return list(map(int, clean_span.split(',')))
  return []

In [ ]:
def find_position(text, entity): #text = sentence where entry is mentioned, entity = what you're trying to find
  text_lower = text.lower() # make lowercase
  entity_lower = entity.lower()
  positions = []
  start = 0
  while start < len(text_lower): #loops when start index within len(text)
    start = text_lower.find(entity_lower, start)
    if start == -1:
      break # entity not found
    end = start + len(entity_lower)
    positions.append([start, end])
    start += len(entity_lower) #update start index, find next entity
  return positions

In [ ]:
def process_row(row):
  source_span = parse_span(row['source_span'])
  target_span = parse_span(row['target_span'])

  if not source_span and row['source_name']: # source_span empty, source_name present
    positions = find_position(row['sentence'], row['source_name'])
    if len(positions) > 1:
      source_span = positions[0] # set to first position found

  if not target_span and row['target_name']:
    positions = find_position(row['sentence'], row['target_name'])
    if len(positions) > 1:
      target_span = positions[0]

  if not source_span:   #check source and target span not NaN
    source_span = ' '
  if not target_span:
    target_span = ' '

  row['source_span'] = source_span
  row['target_span'] = target_span
  return row

In [ ]:
df = new_data.apply(process_row, axis = 1) #apply to each row
df

,sentence,predicted_relation,source_name,target_name,suggested_source,suggested_target,Effect Classification,Notes,dataset,eval_x.1,eval_y.1,sent_id,pmid,source_uid,target_uid,source_tuis,target_tuis,source_span,target_span,db
0,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,"[384, 395]","[453, 465]",oie_chem_dis
1,". Compared to NEC group, lactoferrin-treated N...",affects,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,GENE01,T046,"[384, 395]","[453, 465]",oie_gene_dis
2,". Compared to NEC group, lactoferrin-treated N...",treats,lactoferrin,inflammation,NaN,NaN,Beneficial,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,6.0,31344710,C0022942-NCBI_gene:4057-PIPA_pep_2921950,C0021368,T123,T046,"[384, 395]","[453, 465]",oie_chem_dis
3,[Absorption of fat from three dietary sources ...,causes,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,"[59, 66]","[75, 83]",oie_chem_dis
4,[Absorption of fat from three dietary sources ...,augments,lactose,diarrhea,NaN,NaN,Adverse,NaN,pubmed_triplets_bio_eval,Ilias Apostolakos,Michalis Hadjisimou,0.0,11791477,FDB021790|NPC58629|LTS0113987|BMDB62054|CID613...,C0011991,T123|T109,T184,"[59, 66]","[75, 83]",oie_chem_dis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,Oral administration of lactic acid bacteria (L...,prevents,lactic acid bacteria,respiratory diseases,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,20163592,C1210581,C0035242,ORG02,T047,"[23, 43]","[119, 139]",NaN
2267,We previously demonstrated that administration...,prevents,Akkermansia muciniphila,obesity,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,2.0,27892954,C1490590,C0028754,ORG02,T047,"[50, 73]","[110, 117]",NaN
2268,The phage can also prevent abscess formation a...,prevents,phage,abscess,NaN,NaN,Beneficial,NaN,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,4.0,17517843,C0004651,C0000833,ORG03,T047,"[4, 9]","[27, 34]",NaN
2269,"Thus, this study addressed whether GMI could b...",prevents,GMI,TBI,GMI,TBI,Beneficial,No results are given,microbe_dis,Konstantinos Tachopoulos,Iwannis Charitos,6.0,35562037,C1005127,C0876926,ORG05,T037,"[35, 38]","[73, 76]",NaN


The lines I changed within the raw data:

line 2: I kept only the first line as it contained both "inflammation" and "lactoferrin" and deleted the following lines.

line 3: I kept the third line as it contained both "inflammation" and "lactoferrin", thus deleting the rest of the lines.

line 4: I kept the fourth line as it contained both "inflammation" and "lactoferrin", thus deleting the rest of the lines.

line 194: I changed the first "Lactose" to "Lactoze" to keep the second "lactose" as the source name.

In [ ]:
def create_flagged_sentence(row):
  sentence = row['sentence']
  source_span = row['source_span']
  target_span = row['target_span']

  if any(tag in sentence for tag in ['[s1]', '[e1]', '[s2]', '[e2]']):
    return sentence

  if source_span != ' ': #source_span present
    source_start, source_end = map(int, source_span)
  else:
    source_start = source_end = None

  if target_span != ' ': #target_span present
    target_start, target_end = map(int, target_span)
  else:
    target_start = target_end = None

  if source_start is not None and target_start is not None:
    if target_start < source_start: # swap positions
      source_start, target_start = target_start, source_start
      source_end, target_end = target_end, source_end

  if source_start is not None and target_start is not None:
    sentence = (
        sentence[:source_start] + '[s1]' +sentence[source_start:source_end] + '[e1]' +
        sentence[source_end:target_start] + '[s2]' + sentence[target_start:target_end] + '[e2]' +
        sentence[target_end:]
    )
  elif source_start is not None: #only source_start present
    sentence = sentence[:source_start] + '[s1]' + sentence[source_start:source_end] + '[e1]' + sentence[source_end:]
  elif target_start is not None:
    sentence = sentence[:target_start] + '[s1]' + sentence[target_start:target_end] + '[e1]' + sentence[target_end:]

  return sentence

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'<.*?>', '', text) # remove html tags
  text = re.sub(r'\S+@\S+', '[email]', text) # replace email
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # remove special characters
  text = ' '.join(text.split()) # remove extra white spaces
  return text

In [ ]:
new_data['sentence'] = new_data['sentence'].apply(clean_text)

In [ ]:
df = new_data.apply(process_row, axis = 1)
df['flagged_sentence'] = df.apply(create_flagged_sentence, axis = 1)
df = df[df['Effect Classification'] != 'Neutral'] #filter out 'neutral' classifications
df = df[df['Effect Classification'] != 'Other']

# load data
#x = df['sentence'].values
x = df['flagged_sentence'].values
y = df['Effect Classification'].values
print(df['Effect Classification'].value_counts())

Effect Classification
Adverse       1269
Beneficial     726
Name: count, dtype: int64


In [ ]:
# train logistic regression model, split TF-IDF features into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 42)

In [ ]:
# transform text data with TF-IDF into numerical features
tfidf = TfidfVectorizer()
x_train_tfidf = tfidf.fit(x_train)
x_train_tfidf = tfidf.transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [ ]:
# train logistic regression model, fitted to training data
model = LogisticRegression()
model.fit(x_train_tfidf, y_train)

LogisticRegression()

In [ ]:
# evaluate model using metrics
# 'weighted': metrics calculated for each class, then averaged
y_pred = model.predict(x_test_tfidf) # predict classifications for the test set

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average = 'weighted')
classification_rep = classification_report(y_test, y_pred)

In [ ]:
# train TF-IDF ngrams
tfidf_ngram = TfidfVectorizer(ngram_range = (1, 2))
x_train_tfidf_ngram = tfidf_ngram.fit(x_train)
x_train_tfidf_ngram = tfidf_ngram.transform(x_train)
x_test_tfidf_ngram = tfidf_ngram.transform(x_test)

In [ ]:
model_ngram = LogisticRegression()
model_ngram.fit(x_train_tfidf_ngram, y_train)

LogisticRegression()

In [ ]:
y_pred_ngram = model_ngram.predict(x_test_tfidf_ngram)

accuracy_ngram = accuracy_score(y_test, y_pred_ngram)
precision_ngram = precision_score(y_test, y_pred_ngram, average = 'weighted')
recall_ngram = recall_score(y_test, y_pred_ngram, average = 'weighted')
f1_ngram = f1_score(y_test, y_pred_ngram, average = 'weighted')
classification_rep_ngram = classification_report(y_test, y_pred_ngram)

In [ ]:
print('TF-IDF MODEL RESULTS:')
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print(classification_rep)
print(' ')
print('TF-IDF (N-GRAMS) MODEL RESULTS:')
print('Accuracy:', accuracy_ngram)
print('Precision:', precision_ngram)
print('Recall:', recall_ngram)
print('F1 Score:', f1_ngram)
print(classification_rep_ngram)

TF-IDF MODEL RESULTS:
Accuracy: 0.8521303258145363
Precision: 0.8621836826171192
Recall: 0.8521303258145363
F1 Score: 0.8450289481906638
              precision    recall  f1-score   support

     Adverse       0.83      0.97      0.89       254
  Beneficial       0.92      0.65      0.76       145

    accuracy                           0.85       399
   macro avg       0.87      0.81      0.83       399
weighted avg       0.86      0.85      0.85       399

 
TF-IDF (N-GRAMS) MODEL RESULTS:
Accuracy: 0.8521303258145363
Precision: 0.8638498419962952
Recall: 0.8521303258145363
F1 Score: 0.8445668550088732
              precision    recall  f1-score   support

     Adverse       0.83      0.97      0.89       254
  Beneficial       0.93      0.64      0.76       145

    accuracy                           0.85       399
   macro avg       0.88      0.81      0.83       399
weighted avg       0.86      0.85      0.84       399



In [ ]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("michiyasunaga/BioLinkBERT-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/447k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenizer.add_tokens(['[s1]', '[s2]', '[e1]', '[e2]'])

4

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
y_train_encoded = [1 if i =='Beneficial' else 0 for i in y_train]
y_test_encoded = [1 if i =='Beneficial' else 0 for i in y_test]

In [ ]:
device = 'cuda'

In [ ]:
from transformers import EarlyStoppingCallback, IntervalStrategy
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

train_y = torch.tensor(y_train_encoded, dtype=torch.long).to(device)
test_y = torch.tensor(y_test_encoded, dtype=torch.long).to(device)

In [ ]:
encodings_train = tokenizer(list(x_train), truncation=True, max_length=512)
encodings_test = tokenizer(list(x_test), truncation=True, max_length=512)

In [ ]:
train_dataset = Dataset(encodings_train, train_y)
test_dataset = Dataset(encodings_test, test_y)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    #"distilbert/distilbert-base-uncased",
    "michiyasunaga/BioLinkBERT-base",
    num_labels=2
)

config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.resize_token_embeddings(
        len(tokenizer)
    )

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(28899, 768, padding_idx=0)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer,)

In [ ]:
epochs = 2
training_args = TrainingArguments(
      output_dir=f"checkpoints",
      logging_dir="./logs",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      logging_strategy="epoch",
      num_train_epochs=epochs,
      learning_rate=5e-5,
      per_device_train_batch_size=8,  # 4 in michiyasunaga/BioLinkBERT-large
      per_device_eval_batch_size=8,
      warmup_ratio=0.1,
      weight_decay=0.01,
      adam_beta1=0.9,
      adam_beta2=0.999,
      adam_epsilon=1e-8,
      max_grad_norm=1,
      lr_scheduler_type="linear",
      load_best_model_at_end=True,
      seed=42,
      save_total_limit=2,
  )

trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=train_dataset,
      data_collator=data_collator,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
  )


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
pred = trainer.predict(test_dataset)

labels = pred.label_ids

In [ ]:
import scipy
probs = scipy.special.softmax(pred.predictions, axis=-1)[:,1]
preds = [1 if prob >= 0.5 else 0 for prob in probs]

In [ ]:
accuracy_ngram = accuracy_score(labels, preds)
precision_ngram = precision_score(labels, preds, average = 'weighted')
recall_ngram = recall_score(labels, preds, average = 'weighted')
f1_ngram = f1_score(labels, preds, average = 'weighted')
classification_rep_ngram = classification_report(labels, preds)

In [ ]:
# without special tokens and 5 epochs Distilbert
print('Accuracy:', accuracy_ngram)
print('Precision:', precision_ngram)
print('Recall:', recall_ngram)
print('F1 Score:', f1_ngram)
print(classification_rep_ngram)

Distilbert 5epochs no special tokens
TF-IDF (N-GRAMS) MODEL RESULTS:
Accuracy: 0.9197994987468672
Precision: 0.9200808142805994
Recall: 0.9197994987468672
F1 Score: 0.9199156589833998
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       254
           1       0.88      0.90      0.89       145

    accuracy                           0.92       399
   macro avg       0.91      0.91      0.91       399
weighted avg       0.92      0.92      0.92       399

In [ ]:
print('With tokens in tokenizer and 5 epochs:')
print('Accuracy:', accuracy_ngram)
print('Precision:', precision_ngram)
print('Recall:', recall_ngram)
print('F1 Score:', f1_ngram)
print(classification_rep_ngram)

Distilbert With tokens in tokenizer and 5 epochs:
Accuracy: 0.9172932330827067
Precision: 0.9177583508680176
Recall: 0.9172932330827067
F1 Score: 0.9174709829011077
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       254
           1       0.88      0.90      0.89       145

    accuracy                           0.92       399
   macro avg       0.91      0.91      0.91       399
weighted avg       0.92      0.92      0.92       399

In [ ]:
print('BioLinkBERT With tokens in tokenizer and 5 epochs:')
print('Accuracy:', accuracy_ngram)
print('Precision:', precision_ngram)
print('Recall:', recall_ngram)
print('F1 Score:', f1_ngram)
print(classification_rep_ngram)

change table where it says 0 and 1, 1 change to beneficial and 0 is adverse